# Modeling with Gurobi -- Large Scale Problems

In the first tutorial you have seen the basics of the interaction with Gurobi. We will now look at more realistic interactions such as creating multidimensional and sparse arrays of variables and constraints. Gurobi provides specialized data structures to efficiently handle these situations. Before moving further into this tutorial please go back to the [Python tutorial](../python_tutorial.ipynb) and refresh your mind on *lists*, *dictionaries*, *tuples* and *comprehension*.

## Adding several decision variables

In the first tutorial we have seen that we can add one decision variable to the model using the [method `addVar`](https://www.gurobi.com/documentation/9.0/refman/py_model_addvar.html) of the class `Model`. If we had several variables of the same type we could of course call the `addVar` method several times to create the each individual variable. For example, let $x_i$ for $i=1,\ldots,N$ be binary variables we wish to add to our model. One possibility is the following

In [2]:
from gurobipy import *
N = 100
m = Model('our_model')
for i in range(N):
    m.addVar(vtype=GRB.BINARY,name = "x_"+str(i))
m.update()
print(m)

Academic license - for non-commercial use only
<gurobi.Model MIP instance our_model: 0 constrs, 100 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


This way of adding decision variables is, however, inefficient. In the [details of the class `Model`](https://www.gurobi.com/documentation/9.0/refman/py_model.html) you will notice that the class `Model` has also a [method `addVars`](https://www.gurobi.com/documentation/9.0/refman/py_model_addvars.html) (notice the plural in Var**s**). This method, which permits to add several decision variables to the model, is defined as follows.

`addVars ( *indices, lb=0.0, ub=GRB.INFINITY, obj=0.0, vtype=GRB.CONTINUOUS, name="" )`

It is very similar to the `addVar` method, but it takes as first argument the indices of the decision variables. The indices can be specified in different ways, as we will shortly see. 

### Specifying the dimensions of the variables

The simplest way of specifying the indexes is to provide one or more integers which indicate each dimension. 
As an example, we could add the $x_i$ variables introduced above as follows

In [3]:
m = Model('our_model')
N = 100
x = m.addVars(N,vtype=GRB.BINARY,name = "x")
m.update()
print(m)

<gurobi.Model MIP instance our_model: 0 constrs, 100 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


If we print $x$ we will see a dictionary which has the indices as keys and the variables as values.
In this way we can access each individual variable as with an ordinary dictionary. More precisely, it is a special dictionary provided by Gurobi and named **tupledict**, which we will introduce later.

In [9]:
print(x)
print(x[0]) # The first decision variable
print(x[10]) # The tenth decision variable

{0: <gurobi.Var x[0]>, 1: <gurobi.Var x[1]>, 2: <gurobi.Var x[2]>, 3: <gurobi.Var x[3]>, 4: <gurobi.Var x[4]>, 5: <gurobi.Var x[5]>, 6: <gurobi.Var x[6]>, 7: <gurobi.Var x[7]>, 8: <gurobi.Var x[8]>, 9: <gurobi.Var x[9]>, 10: <gurobi.Var x[10]>, 11: <gurobi.Var x[11]>, 12: <gurobi.Var x[12]>, 13: <gurobi.Var x[13]>, 14: <gurobi.Var x[14]>, 15: <gurobi.Var x[15]>, 16: <gurobi.Var x[16]>, 17: <gurobi.Var x[17]>, 18: <gurobi.Var x[18]>, 19: <gurobi.Var x[19]>, 20: <gurobi.Var x[20]>, 21: <gurobi.Var x[21]>, 22: <gurobi.Var x[22]>, 23: <gurobi.Var x[23]>, 24: <gurobi.Var x[24]>, 25: <gurobi.Var x[25]>, 26: <gurobi.Var x[26]>, 27: <gurobi.Var x[27]>, 28: <gurobi.Var x[28]>, 29: <gurobi.Var x[29]>, 30: <gurobi.Var x[30]>, 31: <gurobi.Var x[31]>, 32: <gurobi.Var x[32]>, 33: <gurobi.Var x[33]>, 34: <gurobi.Var x[34]>, 35: <gurobi.Var x[35]>, 36: <gurobi.Var x[36]>, 37: <gurobi.Var x[37]>, 38: <gurobi.Var x[38]>, 39: <gurobi.Var x[39]>, 40: <gurobi.Var x[40]>, 41: <gurobi.Var x[41]>, 42: <gurobi

Of course, in the same way we can create variables with multiple dimensions. For example, let $y_{ijk}$ be a continuous variable, with $i=1,\ldots,5$, $j=1,\ldots,3$ and $k=1,\ldots,2$

In [13]:
y = m.addVars(5,3,2,name = "y")
m.update()
print(y)
print(y[0,2,1]) # Print y_1,3,2

{(0, 0, 0): <gurobi.Var y[0,0,0]>, (0, 0, 1): <gurobi.Var y[0,0,1]>, (0, 1, 0): <gurobi.Var y[0,1,0]>, (0, 1, 1): <gurobi.Var y[0,1,1]>, (0, 2, 0): <gurobi.Var y[0,2,0]>, (0, 2, 1): <gurobi.Var y[0,2,1]>, (1, 0, 0): <gurobi.Var y[1,0,0]>, (1, 0, 1): <gurobi.Var y[1,0,1]>, (1, 1, 0): <gurobi.Var y[1,1,0]>, (1, 1, 1): <gurobi.Var y[1,1,1]>, (1, 2, 0): <gurobi.Var y[1,2,0]>, (1, 2, 1): <gurobi.Var y[1,2,1]>, (2, 0, 0): <gurobi.Var y[2,0,0]>, (2, 0, 1): <gurobi.Var y[2,0,1]>, (2, 1, 0): <gurobi.Var y[2,1,0]>, (2, 1, 1): <gurobi.Var y[2,1,1]>, (2, 2, 0): <gurobi.Var y[2,2,0]>, (2, 2, 1): <gurobi.Var y[2,2,1]>, (3, 0, 0): <gurobi.Var y[3,0,0]>, (3, 0, 1): <gurobi.Var y[3,0,1]>, (3, 1, 0): <gurobi.Var y[3,1,0]>, (3, 1, 1): <gurobi.Var y[3,1,1]>, (3, 2, 0): <gurobi.Var y[3,2,0]>, (3, 2, 1): <gurobi.Var y[3,2,1]>, (4, 0, 0): <gurobi.Var y[4,0,0]>, (4, 0, 1): <gurobi.Var y[4,0,1]>, (4, 1, 0): <gurobi.Var y[4,1,0]>, (4, 1, 1): <gurobi.Var y[4,1,1]>, (4, 2, 0): <gurobi.Var y[4,2,0]>, (4, 2, 1): <g

### Specifying indices as lists

Another possibility is that of **providing the indices as lists**. For example, consider a decision variable $x_{ij}$ which indicates the amount shipped from city $i$ to warehouse $j$. Given the list of cities and warehouses we can add variables for each pair city-warehouse as follows

In [7]:
m = Model('our_model')
cities = ['CPH','AAR','ALB','ODE']
warehouses = [1,2,3] 
x = m.addVars(cities,warehouses,name = "x")
m.update()
print(m)

<gurobi.Model Continuous instance our_model: 0 constrs, 12 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


In [9]:
print(x)
print(x['CPH',2])
print(x['ALB',3])
# Indices can also be passed as tuples (notice the extra parentheses)
print(x[('CPH',2)])
print(x[('ALB',3)])

{('CPH', 1): <gurobi.Var x[CPH,1]>, ('CPH', 2): <gurobi.Var x[CPH,2]>, ('CPH', 3): <gurobi.Var x[CPH,3]>, ('AAR', 1): <gurobi.Var x[AAR,1]>, ('AAR', 2): <gurobi.Var x[AAR,2]>, ('AAR', 3): <gurobi.Var x[AAR,3]>, ('ALB', 1): <gurobi.Var x[ALB,1]>, ('ALB', 2): <gurobi.Var x[ALB,2]>, ('ALB', 3): <gurobi.Var x[ALB,3]>, ('ODE', 1): <gurobi.Var x[ODE,1]>, ('ODE', 2): <gurobi.Var x[ODE,2]>, ('ODE', 3): <gurobi.Var x[ODE,3]>}
<gurobi.Var x[CPH,2]>
<gurobi.Var x[ALB,3]>
<gurobi.Var x[CPH,2]>
<gurobi.Var x[ALB,3]>


### Specifying indices as lists of tuples

When the variables are specified only for selected tuples of indices, **indices can be provided as lists of tuples**.
For example, assume in the previous example, no shipment can take place between Copenhagen and warehouse number 2, and between Århus and warehouse 1.

In [10]:
m = Model('our_model')
tuples = [('CPH',1),('CPH',3),('AAR',2),('AAR',3),('ALB',1),('ALB',2),('ALB',3),('ODE',1),('ODE',2),('ODE',3)]
x = m.addVars(tuples,name = "x")
m.update()
print(m)

<gurobi.Model Continuous instance our_model: 0 constrs, 10 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


In [11]:
print(x)

{('CPH', 1): <gurobi.Var x[CPH,1]>, ('CPH', 3): <gurobi.Var x[CPH,3]>, ('AAR', 2): <gurobi.Var x[AAR,2]>, ('AAR', 3): <gurobi.Var x[AAR,3]>, ('ALB', 1): <gurobi.Var x[ALB,1]>, ('ALB', 2): <gurobi.Var x[ALB,2]>, ('ALB', 3): <gurobi.Var x[ALB,3]>, ('ODE', 1): <gurobi.Var x[ODE,1]>, ('ODE', 2): <gurobi.Var x[ODE,2]>, ('ODE', 3): <gurobi.Var x[ODE,3]>}


This is applicable also when there are more than two indices as in 

In [13]:
m = Model('our_model')
tuples = [('CPH',1,"A"),('CPH',3,"B"),('AAR',2,"A"),('AAR',3,"C")]
x = m.addVars(tuples,name = "x")
m.update()
print(x)
print(x['CPH',3,'B'])

{('CPH', 1, 'A'): <gurobi.Var x[CPH,1,A]>, ('CPH', 3, 'B'): <gurobi.Var x[CPH,3,B]>, ('AAR', 2, 'A'): <gurobi.Var x[AAR,2,A]>, ('AAR', 3, 'C'): <gurobi.Var x[AAR,3,C]>}
<gurobi.Var x[CPH,3,B]>


But if we try to access `x` for a missing key, such as `('CPH',2,'B')`, we get an error

In [14]:
print(x['CPH',2,'B'])

KeyError: ('CPH', 2, 'B')

### A note on the variable's objective coefficient

The methods `addVar` and `addVars` both allow specifying the objective coefficient of a decision variable when the decision variable is created through the argument `obj`. This can be done in several ways as explained [here](https://www.gurobi.com/documentation/9.0/refman/py_model_addvars.html). Nevertheless, for the sake of clarity, in this course we will always create the objective function separately. 

## Adding several constraints

Very often, constrains have to be specified for a set of indices. Also in this case adding one constraint at a time using method `addConstr` is inefficient. Instead, Gurobi provides the [method `addConstrs`](https://www.gurobi.com/documentation/9.0/refman/py_model_addconstrs.html) (notice the pluaral). The method is specified as follows:

`addConstrs ( generator, name="" ) `

The first argument is a *generator expression*. A generator expression is a Python construct that allows generating *Iterable* objects, that is objects that can be iterated over, such as lists. In our case, we will use generator expressions to build lists of constraints for each specified index. 

As an example, assume we have the following decision variables.

In [27]:
m = Model('our_model')
indices = ['CPH','AAR','ALB','ODE']
x = m.addVars(indices,name = "x")
m.update()
print(m)
print(x)

<gurobi.Model Continuous instance our_model: 0 constrs, 4 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>
{'CPH': <gurobi.Var x[CPH]>, 'AAR': <gurobi.Var x[AAR]>, 'ALB': <gurobi.Var x[ALB]>, 'ODE': <gurobi.Var x[ODE]>}


For each of these variables we have an upper-bound (e.g., a maximum capacity) and we want to add constraints that set the bound on each of these variables. We do it as follows

In [28]:
upper_bound = {'CPH':10,'AAR':20,'ALB':50,'ODE':21}
c = m.addConstrs((x[i] <= upper_bound[i] for i in indices))
m.update()
print(m)
print(c)

<gurobi.Model Continuous instance our_model: 4 constrs, 4 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>
{'CPH': <gurobi.Constr R0>, 'AAR': <gurobi.Constr R1>, 'ALB': <gurobi.Constr R2>, 'ODE': <gurobi.Constr R3>}


In this case the generator expression `(x[i] <= upper_bound[i] for i in indices)` generates a list of constraints of type $x_i \leq upperbound_i$, one for each $i \in indices$.

The generator expression, which must always be provided in parentheses when specifying the name of the constraint, can be arbitrarily complex. It can, of course, include several decision variables

In [2]:
from gurobipy import *
m = Model('our_model')
# We create a dictionary of upper bounds and two sets of indices
upper_bound = {'CPH':10,'AAR':20,'ALB':50,'ODE':21}
cities_dk = ['CPH','AAR','ALB','ODE']
cities_ud = ['STO','OSL','BER']
# We create two sets of variables, namely x and y
x = m.addVars(cities_dk,name = "x")
y = m.addVars(cities_ud,name = "y")
# We add constraints for each pair of indices, involving the two sets of variables
m.addConstrs((x[i] - y[j] <= upper_bound[i] for i in cities_dk for j in cities_ud))
m.update()
print(m)

<gurobi.Model Continuous instance our_model: 12 constrs, 7 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


as well as conditional statements over the indices.

In [3]:
# We add the following constraints for all pairs of cities excluding "CPH" and "STO"
c = m.addConstrs((x[i] + y[j] >= 0 for i in cities_dk for j in cities_ud if (i != "CPH" and j != "STO")))
m.update()
print(m)
print(c)

<gurobi.Model Continuous instance our_model: 18 constrs, 7 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>
{('AAR', 'OSL'): <gurobi.Constr R12>, ('AAR', 'BER'): <gurobi.Constr R13>, ('ALB', 'OSL'): <gurobi.Constr R14>, ('ALB', 'BER'): <gurobi.Constr R15>, ('ODE', 'OSL'): <gurobi.Constr R16>, ('ODE', 'BER'): <gurobi.Constr R17>}


Note that the condition on the indices is specified using the Python `if` statement. If more than one condition are imposed, they must be wrapped by parentheses, as in the example above, `if (condition1 and condition2 not condition3 ...)`.

Very often, the linear expression that make up constraints sum several decision variables. As an example we may wish to specify a bound on some weighed sum of the $x$ variables, such as $\sum_{i\in \mathcal{I}}A_ix_i\leq B$. If the set of $\mathcal{I}$ is small enough, it can be done manually. However, if this set is large, e.g., with dozens of elements, writing this sum becomes impractical. To solve this problem, we can use two techniques: building one constraint, and its linear expression, at a time, or using Gurobi's [global function `quicksum`](https://www.gurobi.com/documentation/9.0/refman/py_quicksum.html).

In [3]:
from gurobipy import *
m = Model('our_model')
# We create a large set of indices and a dictionary of coefficients.
I = range(100)
A = {i: i**2 for i in I}
B = 10200
# We create x
x = m.addVars(I,name = "x")
# We add constraint by building its left-hand-side linear expression first
lhs = 0
for i in I:
    lhs = lhs + A[i] * x[i]
c = m.addConstr(lhs <= B, name = 'bound')
m.update()
print(c)
print(m)

<gurobi.Constr bound>
<gurobi.Model Continuous instance our_model: 1 constrs, 100 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>


The alternative way is to use `quicksum`. This function accepts a list of terms to add (such as linear expressions or decision variables) and returns a linear expression that sums all of them. For example

In [4]:
list_of_terms = [2 * x[0],20 * x[10], 5 * x[7]]
expr = quicksum(list_of_terms)
print(expr)

<gurobi.LinExpr: 2.0 x[0] + 20.0 x[10] + 5.0 x[7]>


In general, it is convenient to build the list of terms using Python's list comprehension (see the Python tutorial if you have doubts). For example, we can create the same constraint as above as follows

In [5]:
terms = [A[i]*x[i] for i in I]
print(terms)
c = m.addConstr(quicksum(terms) <= B, name = 'bound_2')
m.update()
print(c)
print(m)

[<gurobi.LinExpr: 0.0 x[0]>, <gurobi.LinExpr: x[1]>, <gurobi.LinExpr: 4.0 x[2]>, <gurobi.LinExpr: 9.0 x[3]>, <gurobi.LinExpr: 16.0 x[4]>, <gurobi.LinExpr: 25.0 x[5]>, <gurobi.LinExpr: 36.0 x[6]>, <gurobi.LinExpr: 49.0 x[7]>, <gurobi.LinExpr: 64.0 x[8]>, <gurobi.LinExpr: 81.0 x[9]>, <gurobi.LinExpr: 100.0 x[10]>, <gurobi.LinExpr: 121.0 x[11]>, <gurobi.LinExpr: 144.0 x[12]>, <gurobi.LinExpr: 169.0 x[13]>, <gurobi.LinExpr: 196.0 x[14]>, <gurobi.LinExpr: 225.0 x[15]>, <gurobi.LinExpr: 256.0 x[16]>, <gurobi.LinExpr: 289.0 x[17]>, <gurobi.LinExpr: 324.0 x[18]>, <gurobi.LinExpr: 361.0 x[19]>, <gurobi.LinExpr: 400.0 x[20]>, <gurobi.LinExpr: 441.0 x[21]>, <gurobi.LinExpr: 484.0 x[22]>, <gurobi.LinExpr: 529.0 x[23]>, <gurobi.LinExpr: 576.0 x[24]>, <gurobi.LinExpr: 625.0 x[25]>, <gurobi.LinExpr: 676.0 x[26]>, <gurobi.LinExpr: 729.0 x[27]>, <gurobi.LinExpr: 784.0 x[28]>, <gurobi.LinExpr: 841.0 x[29]>, <gurobi.LinExpr: 900.0 x[30]>, <gurobi.LinExpr: 961.0 x[31]>, <gurobi.LinExpr: 1024.0 x[32]>, <gu

By incrementally building linear expressions or using `quicksum` it is possible to create generally complex expressions. In addition, Gurobi provides two useful classes named `tuplelist` and `tupledict` which can help us selecting specific decision variables and indices when building constraints.

## Gurobi `tuplelist`

A Gurobi `tuplelist` is a *sub-class* of the Python list class, that is, it inherits all the functionalities of Python lists, and adds a few extra functionalities which can be quite useful in when building optimization models.  Particularly, when a `tuplelist` is populated with a list of tuples, the `select` method efficiently selects tuples whose values match specified values. You should familiarize with the documentation [here](https://www.gurobi.com/documentation/9.0/refman/py_tuplelist.html).

One builds a `tuplelist` by passing a list of tuples

In [16]:
l = tuplelist([(1, 2), (1, 3), (2, 3), (2, 4)])
print(l)

<gurobi.tuplelist (4 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 2 , 3 )
 ( 2 , 4 )
>


Similarly to an ordinary list, you can for example append new tuples to the list and concatenate lists, as well as call the methods `append`, `extend`, `insert`, `pop`, and `remove`.


In [17]:
l.append((3,4))
print(l)

<gurobi.tuplelist (5 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 2 , 3 )
 ( 2 , 4 )
 ( 3 , 4 )
>


To access the members of a `tuplelist`, you also use standard list functions. For example, `l[0]` returns the first member of a tuplelist, while `l[0:3]` returns a `tuplelist` containing the first three members. You can also use `len(l)` to query the length of a list.

In [19]:
print(l[0])
print(l[0:3])
print(len(l))

(1, 2)
[(1, 2), (1, 3), (2, 3)]
5


Perhaps the most special feature of `tuplelist` is the `select` method which allows us to select a sub-list where particular tuple entries match desired values. The reference of the method is provided [here](https://www.gurobi.com/documentation/9.0/refman/py_tuplelist_select.html).
One needs to pass to the select method the pattern that must be matched by the tuples to select. 
The pattern includes a value for each element of the tuples included in the list, thus the number of arguments to the `select` method is equal to the number of entries in the members of the `tuplelist`. A `'*'` string indicates that any value is acceptable.

In the example above, `l` contains tuples with two entries, so we can select elements perform the following selections:

In [21]:
# All tuples with 1 for the first element
print(l.select(1,'*'))
# All tuples with 3 as the second element
print(l.select('*',3))
# All tuples with 5 as the second element (none in this example)
print(l.select('*',5))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
>
<gurobi.tuplelist (2 tuples, 2 values each):
 ( 1 , 3 )
 ( 2 , 3 )
>
[]


You can also provide a list argument to indicate that multiple values are acceptable for a given position in the tuple

In [22]:
# All tuples with either 1 or 2 for the first element
print(l.select([1,2],'*'))
# All tuples with eith 3 or 4 as the second element
print(l.select('*',[3,4]))

<gurobi.tuplelist (4 tuples, 2 values each):
 ( 1 , 2 )
 ( 1 , 3 )
 ( 2 , 3 )
 ( 2 , 4 )
>
<gurobi.tuplelist (4 tuples, 2 values each):
 ( 1 , 3 )
 ( 2 , 3 )
 ( 2 , 4 )
 ( 3 , 4 )
>


Obviously, `tuplelists` can host tuples with more than two (not necessarily numerical) elements

In [23]:
l = tuplelist([(1, 2,'a','k'), (1, 3,'b','k'), (2, 3,'n','z'), (2, 4,'m','z')])
print(l)
print(l.select('*','*','*','z'))
print(l.select('*','*',['a','n'],'*'))

<gurobi.tuplelist (4 tuples, 4 values each):
 ( 1 , 2 , a , k )
 ( 1 , 3 , b , k )
 ( 2 , 3 , n , z )
 ( 2 , 4 , m , z )
>
<gurobi.tuplelist (2 tuples, 4 values each):
 ( 2 , 3 , n , z )
 ( 2 , 4 , m , z )
>
<gurobi.tuplelist (2 tuples, 4 values each):
 ( 1 , 2 , a , k )
 ( 2 , 3 , n , z )
>


Gurobi `tuplelist`s can be very useful for creating spare arrays of variables and constraints. For example, we may want to create decision variables only for the tuples with 'a' or 'b' in the third position

In [30]:
m = Model('our_model')
x = m.addVars(l.select('*','*',['a','b'],'*'),name = "x")
print(m)
print(x)

<gurobi.Model Continuous instance our_model: 0 constrs, 2 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>
{(1, 2, 'a', 'k'): <gurobi.Var x[1,2,a,k]>, (1, 3, 'b', 'k'): <gurobi.Var x[1,3,b,k]>}


Note that we could achieve the same result using plain Python list comprehension

In [31]:
y = m.addVars([(i,j,n,m) for i,j,n,m in l if n in ['a','b']],name = "y")
m.update()
print(y)

{(1, 2, 'a', 'k'): <gurobi.Var y[1,2,a,k]>, (1, 3, 'b', 'k'): <gurobi.Var y[1,3,b,k]>}


However, the latter statement considers every member in the list and then makes a selection based on the `if` statement, which can be quite inefficient for large lists. The select method, on the other hand, builds internal data structures that make these selections quite efficient.

Similarly, we can use `select` to specify for which elements one must create constraints

In [27]:
c = m.addConstrs((x[i,j,n,m] <= 1 for i,j,n,m in l.select('*','*',['a','b'],'*')))
m.update()
print(m)
print(c)

<gurobi.Model Continuous instance our_model: 2 constrs, 2 vars, Parameter changes: LogFile=gurobi.log, CSIdleTimeout=1800>
{(1, 2, 'a', 'k'): <gurobi.Constr R0>, (1, 3, 'b', 'k'): <gurobi.Constr R1>}


## Gurobi `tupledict`

A `tupledic` is a sub-class of the Python `dict` class mainly designed to efficiently work with subsets of Gurobi variable objects. Being it a sub-class of `dict`, it inherits all methods of ordinary Python dictionaries. In addition, it provides methods for summing and multiplying that can be used to easily and concisely build linear expressions that include only a subset of the decision variables. 

In particular, a `tupledict` is a Python dict where the keys are stored as a Gurobi `tuplelist`, and where the values are typically Gurobi decision variables, that is [Var objects](https://www.gurobi.com/documentation/9.0/refman/py_var.html#pythonclass:Var). Objects of this class make it easier to build linear expressions which include sets of Gurobi variables, using the `tuplelist.select()` syntax and semantics. 

While you can build your own `tupledict`, the [`addVars` method](https://www.gurobi.com/documentation/9.0/refman/py_model_addvars.html#pythonmethod:Model.addVars) adds one Gurobi decision variable to the model for each tuple in the input argument(s) and returns a `tupledict` where the keys are the tuples provided, and the values are the decision variables.

In [3]:
from gurobipy import *
m = Model('our_model')
# These are the tuples for which we want to create decision variables
l = list([(1, 2), (1, 3), (2, 3), (2, 4)])
# We add decision variables to the model. 
# This method returns a tupledic
x = m.addVars(l, name="x")
m.update()
# x will be a tupledict with one variable for each tuple in the list l
print(x)
print(type(x))

{(1, 2): <gurobi.Var x[1,2]>, (1, 3): <gurobi.Var x[1,3]>, (2, 3): <gurobi.Var x[2,3]>, (2, 4): <gurobi.Var x[2,4]>}
<class 'gurobipy.tupledict'>


The resulting dictionary has the following structure

| Key           | Value          |
| ------------- |:-------------: | 
| (1,2)         | $x_{1,2}$      |
| (1,3)         | $x_{1,3}$      |
| (2,3)         | $x_{2,3}$      |
| (2,4)         |$x_{2,4}$       |

The `tupledict` class has two methods worth of notice (see the documentation [here](https://www.gurobi.com/documentation/9.0/refman/py_tupledict.html)), namely `sum` and `prod` (in addition to a `select` method that works as for `tuplelist`).

The [`sum` method](https://www.gurobi.com/documentation/9.0/refman/py_tupledict_sum.html) can be used to sum subsets of the decision variables defined by specific criteria. As an example we may wish to sum all decision variables which have $1$ as the first index, that is $x_{1,2}$ and $x_{1,3}$. We do this as follows

In [7]:
expr = x.sum(1,'*')
print(expr)

<gurobi.LinExpr: x[1,2] + x[1,3]>


The method sum will create a linear expression which sums the variables specified in the argument. Another example: sum all $x$ variables having $3$ as the second index

In [8]:
expr = x.sum('*',3)
print(expr)

<gurobi.LinExpr: x[1,3] + x[2,3]>


Note that when using the `sum` method all variables appear with a coefficient of $1$. For the cases when the variables have coefficients other than $1$ `tupledict` has the [`prod` method](https://www.gurobi.com/documentation/9.0/refman/py_tupledict_prod.html).  Coefficients are provided through a `dict` argument and are indexed using the same tuples as the `tupledict`. 

As an example, one can sum all decision variables multiplied by their coefficients

In [9]:
coefficients = {(1, 2): 3, (1, 3): 5.2, (2, 3): 1.8, (2, 4):7.2}
expr = x.prod(coefficients)
print(expr)

<gurobi.LinExpr: 3.0 x[1,2] + 5.2 x[1,3] + 1.8 x[2,3] + 7.2 x[2,4]>


Also in this case we can select specific subsets of the variables. In this case the variables are specified after the coefficients

In [10]:
expr1 = x.prod(coefficients,1,'*')
print(expr1)
expr2 = x.prod(coefficients,'*',4)
print(expr2)

<gurobi.LinExpr: 3.0 x[1,2] + 5.2 x[1,3]>
<gurobi.LinExpr: 7.2 x[2,4]>


Finally, notice that also the `addConstrs` method returns a `tupledict` which associates a constraint to each tuple of indices for which it is defined.

In [1]:
from gurobipy import *
m = Model('our_model')
# We create a dictionary of upper bounds and two sets of indices
upper_bound = {'CPH':10,'AAR':20,'ALB':50,'ODE':21}
cities_dk = ['CPH','AAR','ALB','ODE']
cities_ud = ['STO','OSL','BER']
# We create two sets of variables, namely x and y
x = m.addVars(cities_dk,name = "x")
y = m.addVars(cities_ud,name = "y")
# The addConstrs method return a tupledict
c = m.addConstrs(sum(x.select('*')) - y[j] <= 10 for j in cities_ud)
m.update()
print(c)
print(type(c))

Academic license - for non-commercial use only
{'STO': <gurobi.Constr R0>, 'OSL': <gurobi.Constr R1>, 'BER': <gurobi.Constr R2>}
<class 'gurobipy.tupledict'>


## A complete example

A set $\mathcal{K}$ of commodities must be shipped from producers to warehouses located in different nodes of a network $\mathcal{N}$ to satisfy given demand $D_{ik}$ (negative for warehouses, positive for producers). Each arc $(i,j)$ in the transportation network has a unit cost $C_{ijk}$ associated with it and with each product, and a total capacity $Q_{ij}$. The network contains only the arcs $\set{A}$ between producers and warehouses. Let $x_{ijk}$ be the amount of product $k$ sent on arc $(i,j)$. Find the minimum cost shipment.

The problem can be modeled as follows
$$\min \sum_{(i,j)\in\mathcal{A}}\sum_{k\in\mathcal{K}}C_{ijk}x_{ijk}$$
$$\sum_{k\in\mathcal{K}}x_{ijk}\leq Q_{ij} \qquad (i,j)\in\mathcal{A}$$
$$\sum_{j\in\mathcal{N}}x_{ijk}-\sum_{j\in\mathcal{N}}x_{jik} = D_{ik} \qquad i \in\mathcal{N}, k\in\mathcal{K}$$
$$x_{ijk}\geq 0$$

In [14]:
from gurobipy import *
# Data
products = tuplelist(['Pencils', 'Pens'])
nodes = tuplelist(['Detroit', 'Denver', 'Boston', 'New York', 'Seattle'])
arcs = tuplelist([('Detroit', 'Boston'),
                 ('Detroit', 'New York'),
                 ('Detroit', 'Seattle'),
                 ('Denver',  'Boston'),
                 ('Denver',  'New York'),
                 ('Denver',  'Seattle')])
capacity = {
    ('Detroit', 'Boston'):   100,
    ('Detroit', 'New York'):  80,
    ('Detroit', 'Seattle'):  120,
    ('Denver',  'Boston'):   120,
    ('Denver',  'New York'): 120,
    ('Denver',  'Seattle'):  120}

cost = {
    ('Pencils', 'Detroit', 'Boston'):   10,
    ('Pencils', 'Detroit', 'New York'): 20,
    ('Pencils', 'Detroit', 'Seattle'):  60,
    ('Pencils', 'Denver',  'Boston'):   40,
    ('Pencils', 'Denver',  'New York'): 40,
    ('Pencils', 'Denver',  'Seattle'):  30,
    ('Pens',    'Detroit', 'Boston'):   20,
    ('Pens',    'Detroit', 'New York'): 20,
    ('Pens',    'Detroit', 'Seattle'):  80,
    ('Pens',    'Denver',  'Boston'):   60,
    ('Pens',    'Denver',  'New York'): 70,
    ('Pens',    'Denver',  'Seattle'):  30}

demand = {
    ('Pencils', 'Detroit'):   50,
    ('Pencils', 'Denver'):    60,
    ('Pencils', 'Boston'):   -50,
    ('Pencils', 'New York'): -50,
    ('Pencils', 'Seattle'):  -10,
    ('Pens',    'Detroit'):   60,
    ('Pens',    'Denver'):    40,
    ('Pens',    'Boston'):   -40,
    ('Pens',    'New York'): -30,
    ('Pens',    'Seattle'):  -30}

# Model
m = Model('flow')

# Decision variables
x = m.addVars(products,arcs,name = "x")

# Objective function 
# Remember that in order to use prod the dictionary of coefficients must have the
# same keys as the dictionary of variables
expr = x.prod(cost)
m.setObjective(expr, GRB.MINIMIZE)

# Capacity constraints
c1 = m.addConstrs((x.sum('*',i,j) <= capacity[i,j] for i,j in arcs),name="Cap")

# Demand constraints
c2 = m.addConstrs((x.sum(k,i,'*') - x.sum(k,'*',i) == demand[k,i] for i in nodes for k in products),name="Dem")

m.optimize()

if m.status == GRB.OPTIMAL:
    print("\nTotal cost : %g " % m.ObjVal)
    solution = m.getAttr('x', x)
    for h in products:
        print('\nOptimal flows for %s:' % h)
        for i, j in arcs:
            if solution[h, i, j] > 0:
                print('%s -> %s: %g' % (i, j, solution[h, i, j]))

Optimize a model with 16 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 16 rows and 12 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  5.500000000e+03

Total cost : 5500 

Optimal flows for Pencils:
Detroit -> Boston: 50
Denver -> New York: 50
Denver -> Seattle: 10

Optimal flows for Pens:
Detroit -> Boston: 30
Detroit -> New York: 30
Denver -> Boston: 10
Denver -> Seattle: 30
